In [ ]:
#pip install -U tavily-python langchain_community
from typing import Annotated
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    messages: Annotated[list, add_messages]

 # Import the necessary tool class, in this case TavilySearchResults
tool = TavilySearchResults(max_results=2)
tools = [tool]

# Set the model as ChatOpenAI
llm = ChatOpenAI(temperature=0) 

#Call the model with a user message
# content = llm.invoke(("user", 'Hey there')).content
# content

# Create an instance of the tool and bind it to the LLM using llm.bind_tools(). 
# This informs the LLM about the available tools and their usage.
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools=[tool])

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
# Add the ToolNode to the Graph
graph_builder.add_node("tools", tool_node)

# Utilize graph_builder.add_conditional_edges() to set up routing logic based on whether the LLM decides to call a tool. 
# The tools_condition function checks if the LLM's response includes tool invocation instructions
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

# Create a MemorySaver object to act as the checkpointer
memory = MemorySaver()

# Compile the graph, passing in the 'memory' object as the checkpointer
graph = graph_builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # Process user input through the LangGraph
    for event in graph.stream({"messages": [("user", user_input)]}, config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


User:  集团内


Assistant: 
Assistant: [{"url": "https://www.chinaacc.com/shuishou/dyjh/qi20230213111231.shtml", "content": "问集团内成员单位如何界定，具体包括哪些单位？参考《企业集团财务公司管理办法》（中国银行保险监督管理委员会令2022年第6号）规定：\"第三条 本办法所称企业集团是指在中华人民共和国境内依法登记，以资本为联结纽带、以母子公司为主体、以集团章程为共同行为规范，由母公司、子公司"}, {"url": "https://baike.baidu.com/item/集团公司/6161765", "content": "集团公司是为了一定的目的组织起来共同行动的团体公司，是指以资本为主要联结纽带，以母子公司为主体，以集团章程为共同行为规范的，由母公司、子公司、参股公司及其他成员共同组成的企业法人联合体。一般意义上的集团公司，是指拥有众多生产、经营机构的大型公司。它一般都经营着规模"}]
Assistant: 在企业集团内，成员单位如何界定以及具体包括哪些单位可以参考《企业集团财务公司管理办法》的规定。企业集团是指在中华人民共和国境内依法登记，以资本为联结纽带、以母子公司为主体、以集团章程为共同行为规范，由母公司、子公司共同组成的企业法人联合体。

另外，集团公司是一个为了一定目的而组织起来共同行动的团体公司，以资本为主要联结纽带，以母子公司为主体，以集团章程为共同行为规范，由母公司、子公司、参股公司及其他成员共同组成的企业法人联合体。

如果需要更详细或具体的信息，请访问相关链接：
1. [中国会计网-《企业集团财务公司管理办法》](https://www.chinaacc.com/shuishou/dyjh/qi20230213111231.shtml)
2. [百度百科-集团公司](https://baike.baidu.com/item/集团公司/6161765)


In [5]:
memory